In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import librosa
import librosa.display

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, LeakyReLU
from keras.losses import mse, binary_crossentropy
from keras.utils import np_utils, plot_model
from keras import backend as K
from keras import optimizers
from keras import regularizers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv
/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy


In [2]:
train_data = np.load('/kaggle/input/audio-binary-classification/train_data.npy');
test_data = np.load('/kaggle/input/audio-binary-classification/test_data.npy');


print("inputs loaded")

inputs loaded


In [3]:
#Scale and Reshape
train_data = train_data.astype('float32');
test_data = test_data.astype('float32');

#Normalize Data
#train_data /= np.amax(test_data);
#test_data /= np.amax(test_data);

print(train_data.shape);

train_data = train_data.reshape(-1,44100,1);
test_data = test_data.reshape(-1,44100,1);

input_shape = train_data.shape;
print(input_shape);

(3999, 44100)
(3999, 44100, 1)


In [4]:
#Look at CSV and store as array
train_labels = pd.read_csv("/kaggle/input/audio-binary-classification/train_labels.csv");
train_labels = train_labels.values;
train_labels = train_labels[:,1];
print(train_labels[:10]);
print(train_labels.shape)

[0 0 0 1 0 0 1 0 1 0]
(3999,)


In [5]:
#Modify data
x_data = np.zeros( (train_data.shape[0] * 2 + 250, train_data.shape[1],1) );
y_data = np.zeros( (train_labels.shape[0] * 2 + 250))

x_data[:3999,:,0] = train_data[:3999,:,0];
y_data[:3999] = train_labels[:3999];

for a in range(train_data.shape[0]):
    s = np.random.normal(1,.03,44100);
    x_data[a+3999,:,0] = np.multiply(x_data[a,:,0],s);
    y_data[a+3999] = train_labels[a];
        
b = 0;

for a in range(train_data.shape[0]):
    if(train_labels[a] == 1):
        s = np.random.normal(1,.03,44100);
        x_data[2*3999+b,:,0] = np.multiply(x_data[a,:,0],s);
        y_data[2*3999+b] = train_labels[a];
        b += 1;
    if(b >= 250):
        break;
        
print("data made noisey")
        

data made noisey


In [6]:
def spec(arr):
    sr = 44.1;
    mfcc = librosa.feature.mfcc(y=arr,sr=sr,n_mfcc=40);
    return mfcc;

In [7]:
a = spec(x_data[4,:,0]);
print(a.shape)
x_train = np.zeros( (x_data.shape[0], a.shape[0],a.shape[1],1) );
x_test = np.zeros( (test_data.shape[0], a.shape[0],a.shape[1],1) )

for i in range(x_data.shape[0]):
    x_train[i,:,:,0] = spec(x_data[i,:,0]);

for i in range(x_test.shape[0]):
    x_test[i,:,:,0] = spec(test_data[i,:,0]);
    
print("spectrograms created");

(40, 87)
spectrograms created


In [8]:
# create convolution neural network
model = Sequential()
model.add(Conv2D(164, kernel_size=(5,5),
                 input_shape=(40,87,1)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.45))

model.add(Conv2D(128, (6,6)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.45))

model.add(Conv2D(64, (3,3)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.33))

model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.5))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 36, 83, 164)       4264      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 36, 83, 164)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 41, 164)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 41, 164)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 36, 128)       755840    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 13, 36, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 12, 128)        0         
__________

In [9]:
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='sgd', 
              metrics=['accuracy'])

In [10]:
history = model.fit(x_train,y_data, shuffle=True,
          batch_size=30,epochs=1500,verbose=1)

Epoch 1/1500
8248/8248 [==============================] - 11s 1ms/step - loss: 0.1662 - acc: 0.8281
Epoch 2/1500
8248/8248 [==============================] - 4s 534us/step - loss: 0.1512 - acc: 0.8476
Epoch 3/1500
8248/8248 [==============================] - 4s 537us/step - loss: 0.1509 - acc: 0.8478
Epoch 4/1500
8248/8248 [==============================] - 4s 528us/step - loss: 0.1497 - acc: 0.8497
Epoch 5/1500
8248/8248 [==============================] - 4s 526us/step - loss: 0.1496 - acc: 0.8499
Epoch 6/1500
8248/8248 [==============================] - 4s 530us/step - loss: 0.1497 - acc: 0.8500
Epoch 7/1500
8248/8248 [==============================] - 4s 526us/step - loss: 0.1495 - acc: 0.8497
Epoch 8/1500
8248/8248 [==============================] - 4s 529us/step - loss: 0.1493 - acc: 0.8501
Epoch 9/1500
8248/8248 [==============================] - 4s 524us/step - loss: 0.1501 - acc: 0.8497
Epoch 10/1500
8248/8248 [==============================] - 4s 526us/step - loss: 0.1492 - ac

In [11]:
Y_pred = model.predict(x_test);
print(Y_pred)
df_pred = pd.DataFrame(Y_pred);
df_pred.index.name = "Id"
df_pred = pd.DataFrame.rename(df_pred,columns={0:"Label"})
print(df_pred);
df_pred.to_csv("output.csv")

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [1.]]
      Label
Id         
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
...     ...
3992    0.0
3993    0.0
3994    0.0
3995    0.0
3996    1.0

[3997 rows x 1 columns]
